In [1]:
import openeo
import geopandas as gpd
import pandas as pd
from openeo_classification.landuse_classification import *
from datetime import date
import ipywidgets as widgets
import datetime
import json

#### Target data 
LUCAS, 2018 (land use cover)

#### Input data
From S2: calculation of 7 indices (NDVI, NDMI, NDGI, ANIR, NDRE1, NDRE2, NDRE5) and keeping 2 bands (B06, B12)
From S1: VV, VH and VV/VH
For all of these, 10 features: p25, p50, p75, sd and 6 t-steps, with flexible range

#### Model
Random Forest, trained using custom hyperparameter, 70/30 split

In [2]:
train_test_split, algorithm, nrtrees, mtry, fusion_technique, aoi, strat_layer, include_mixed_pixels, start_date, end_date, nr_targets, nr_spp = getStartingWidgets()

Box(children=(Label(value='Train / test split:'), FloatSlider(value=0.75, max=1.0, step=0.05)))

Dropdown(description='Model:', disabled=True, options=('Random Forest',), value='Random Forest')

Box(children=(Label(value='Hyperparameters RF model:'), IntText(value=250, description='Nr trees:'), IntText(v…

Box(children=(Label(value='S1 / S2 fusion:'), RadioButtons(options=('Feature fusion', 'Decision fusion'), valu…

FileUpload(value={}, accept='.geojson,.shp', description='Upload AOI', layout=Layout(width='20em'))

...

Box(children=(Label(value='Include mixed pixels:'), RadioButtons(options=('Yes', 'No'), value='Yes')))

DatePicker(value=datetime.date(2018, 1, 1), description='Start date')

DatePicker(value=datetime.date(2018, 12, 31), description='End date')

Box(children=(Label(value='Select the amount of target classes:'), IntSlider(value=10, max=37, min=2)))

Box(children=(Label(value='Select the amount of times you want to point sample each reference polygon:'), IntS…

In [3]:
target_classes = getTargetClasses(nr_targets)

SelectMultiple(description='Target class', options=('A00: Artificial land', 'A10: Roofed built-up areas', 'A20…

SelectMultiple(description='Target class', options=('A00: Artificial land', 'A10: Roofed built-up areas', 'A20…

SelectMultiple(description='Target class', options=('A00: Artificial land', 'A10: Roofed built-up areas', 'A20…

In [4]:
y = getReferenceSet(aoi, nr_spp, target_classes)

Loading in the LUCAS Copernicus dataset...
Finished loading data.
Extracting points and converting target labels...
Finished extracting points and converting target labels


In [5]:
def getStrata():
    if len(strat_layer.value) == 0:
        strata = gpd.GeoDataFrame.from_features(json.loads(aoi.data[0]))
    else:
        strata = gpd.GeoDataFrame.from_features(json.loads(strat_layer.data[0]))
    return strata

strata = getStrata()

In [9]:
def fitRandomForestModel(feature_raster, y):
    features, feature_list = load_lc_features(feature_raster, y, start_date.value, end_date.value)
    X = features.aggregate_spatial(json.loads(y.to_json()), reducer="mean")
    ml_model = X.fit_class_random_forest(target=json.loads(y.to_json()), training=train_test_split.value, num_trees=nrtrees.value, mtry=mtry.value)
    model = ml_model.save_ml_model()
    training_job = model.create_job()
    training_job.start_and_wait()
    return training_job.job_id

In [ ]:
jobids = {}
for index, stratum in enumerate(strata["geometry"]):
    y_final = gpd.clip(y, stratum)
    if fusion_technique.value == "Decision fusion":
        jobids["s1_stratum"+str(index)] = fitRandomForestModel("s1", y_final)
        jobids["s2_stratum"+str(index)] = fitRandomForestModel("s2", y_final)
    else:
        jobids["both_stratum"+str(index)] = fitRandomForestModel("both", y_final)

Authenticated using refresh token.
Authenticated using refresh token.
0:00:00 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': send 'start'
0:01:06 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)
0:01:11 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)
0:01:18 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)
0:01:26 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)
0:01:37 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)
0:01:49 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)
0:02:06 Job 'd458856e-b315-4d13-a369-a2c86f3adaf2': queued (progress N/A)


In [14]:
cube = features.filter_spatial(gpd.GeoDataFrame.from_features(json.loads(list(aoi.value.values())[0]["content"]))["geometry"][0])
predicted = cube.predict_random_forest(
    model="f18fe77e-8767-4132-9977-43e13ae33999",
    dimension="bands"
)

nieuwe_job = predicted.execute_batch(format="GTiff")

0:00:00 Job '490c49ec-6cbd-4664-bf79-dc924106cc1a': send 'start'


OpenEoApiError: [500] unknown: Did not find value which can be converted into java.lang.String